<a href="https://colab.research.google.com/github/dhwanipatel14/Comp-4730-FinalProject/blob/master/google_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Required Libraries Import

In [3]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import string
import os
import re
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Embedding, Concatenate, TimeDistributed, Bidirectional,GRU, Flatten,Conv2D,Conv1D,GlobalMaxPooling1D,GlobalMaxPool1D,SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
training_set      = pd.read_csv('train.csv')
testing_set       = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')


In [6]:
pd.set_option('display.max_columns', None)

targets = list(sample_submission.columns[1:])
training_set[targets].describe()


,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000
mean,0.892663,0.595301,0.057301,0.698525,0.772633,0.793689,0.587478,0.507275,0.238745,0.004469,0.429978,0.284915,0.038137,0.010035,0.030762,0.065225,0.497587,0.166063,0.386385,0.000823,0.799931,0.925408,0.654823,0.960054,0.968626,0.854680,0.479547,0.130641,0.502468,0.908254
std,0.132047,0.219470,0.182196,0.350938,0.303023,0.336622,0.135900,0.185987,0.335057,0.045782,0.365952,0.368826,0.153635,0.074240,0.138065,0.197582,0.423138,0.257301,0.383384,0.020489,0.178420,0.114836,0.107666,0.086926,0.074631,0.130743,0.422921,0.225718,0.407097,0.100708
min,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.333333,0.333333,0.200000,0.000000,0.000000,0.000000,0.333333
25%,0.777778,0.444444,0.000000,0.500000,0.666667,0.666667,0.444444,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.888889,0.666667,1.000000,1.000000,0.800000,0.000000,0.000000,0.000000,0.888889
50%,0.888889,0.555556,0.000000,0.666667,1.000000,1.000000,0.555556,0.444444,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.333333,0.000000,0.833333,1.000000,0.666667,1.000000,1.000000,0.866667,0.500000,0.000000,0.500000,0.888889
75%,1.000000,0.777778,0.000000,1.000000,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.000000,1.000000,0.333333,0.666667,0.000000,1.000000,1.000000,0.666667,1.000000,1.000000,0.933333,1.000000,0.333333,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.666667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
print("Training Set Size (rows, cols): ", training_set.shape)

training_set_cols = training_set.columns
n = 1

print("\nLIST OF COLUMN NAMES IN TRAINING SET:")
print("----------------------------------------")
for i in training_set_cols:
    print(n, ". ", i)
    n = n + 1

Training Set Size (rows, cols):  (6079, 41)

LIST OF COLUMN NAMES IN TRAINING SET:
----------------------------------------
1 .  qa_id
2 .  question_title
3 .  question_body
4 .  question_user_name
5 .  question_user_page
6 .  answer
7 .  answer_user_name
8 .  answer_user_page
9 .  url
10 .  category
11 .  host
12 .  question_asker_intent_understanding
13 .  question_body_critical
14 .  question_conversational
15 .  question_expect_short_answer
16 .  question_fact_seeking
17 .  question_has_commonly_accepted_answer
18 .  question_interestingness_others
19 .  question_interestingness_self
20 .  question_multi_intent
21 .  question_not_really_a_question
22 .  question_opinion_seeking
23 .  question_type_choice
24 .  question_type_compare
25 .  question_type_consequence
26 .  question_type_definition
27 .  question_type_entity
28 .  question_type_instructions
29 .  question_type_procedure
30 .  question_type_reason_explanation
31 .  question_type_spelling
32 .  question_well_written
33 . 

In [8]:
print("Testing Set Size (rows, cols): ", testing_set.shape)

testing_set_cols = testing_set.columns
n = 1

print("\nLIST OF COLUMN NAMES IN TESTING SET:")
print("----------------------------------------")
for i in testing_set_cols:
    print(n, ". ", i)
    n = n + 1

Testing Set Size (rows, cols):  (476, 11)

LIST OF COLUMN NAMES IN TESTING SET:
----------------------------------------
1 .  qa_id
2 .  question_title
3 .  question_body
4 .  question_user_name
5 .  question_user_page
6 .  answer
7 .  answer_user_name
8 .  answer_user_page
9 .  url
10 .  category
11 .  host


In [9]:
training_set.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [19]:
import nltk


# for removing the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# for stemming and lemmatization
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
print("RANDOM QUESTION WITHOUT MINING:")
print(training_set['question_body'].values[344])

print("RANDOM ANSWER WITHOUT MINING:")
print(training_set['answer'].values[344])

RANDOM QUESTION WITHOUT MINING
---------------------------------------------------------------
site migrating wordpress need add properties users edit e g address city state business name etc along properties administrators edit isactive canemail displayed user addition need able display properties table similar plugin members list displays custom fields displaying well given requirements attempted use combination cimy user extra fields members list members list grid option display extra fields created plugin would recommend approach edit guess crux question preferred method add properties user 
RANDOM ANSWER WITHOUT MINING
---------------------------------------------------------------
To answer the first part of your question, I have just put my class TTT_User_Profile_Addon on GitHub. The class offers a simple interface to add a field to a profile page. I have added an example for a checkbox subclass and some code to initialize it per functions.php. This works in a plugin too, of cou

In [21]:
def mineText(text):
    text = str(re.sub(r'http\S+', '', text))
    text = str(re.sub(r'<.*?>', ' ', text))
    text = str(text.lower())
    text = str(re.sub(r"\S*\d\S*", "", text))
    text = str(re.sub('[^A-Za-z0-9]+', ' ', text))
    text = str(' '.join([word for word in text.split(' ') if word not in stopwords.words('english')]))
    text = str(' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')]))
    return text

training_set['question_body'] = training_set['question_body'].apply(mineText)
training_set['answer'] = training_set['answer'].apply(mineText)


In [24]:
print("RANDOM QUESTION AFTER MINING:")
print(training_set['question_body'].values[344])

print("\nRANDOM ANSWER AFTER MINING:")

print(training_set['answer'].values[344])


RANDOM QUESTION AFTER MINING:
site migrating wordpress need add property user edit e g address city state business name etc along property administrator edit isactive canemail displayed user addition need able display property table similar plugin member list display custom field displaying well given requirement attempted use combination cimy user extra field member list member list grid option display extra field created plugin would recommend approach edit guess crux question preferred method add property user 

RANDOM ANSWER AFTER MINING:
answer first part question put class ttt user profile addon github class offer simple interface add field profile page added example checkbox subclass code initialize per function php work plugin course build placeholder add separate filter markup input value make extending class easier set custom capability showing saving field per constructor call whole work reduced simple init function add action init ttt init profile addons register extra fiel

In [25]:
# parameters for embeddings
maxLength = 1000
maxFeatures = 5000 
embeddingSize = 768

inp = Input(shape=(maxLength,)) # returns a shape tuple of ints, size of the maxLength  

z = Embedding(maxFeatures,embeddingSize,input_length = maxLength)(inp)
z = Bidirectional(SimpleRNN(60,return_sequences='True'))(z)
z = GlobalMaxPool1D()(z)
z = Dense(16,activation='relu')(z)
z = Dense(5,activation='softmax')(z)

model = Model(inputs=inp,outputs=z)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 embedding (Embedding)       (None, 1000, 768)         3840000   
                                                                 
 bidirectional (Bidirectiona  (None, 1000, 120)        99480     
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 120)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 16)                1936      
                                                                 
 dense_1 (Dense)             (None, 5)                 85    

In [28]:
yLabel = LabelEncoder() # label encodeer
labels = yLabel.fit_transform(training_set['category']) # used for the labels of each category

yTrain = labels # set x training vars as the labels (categories)
# setup the training (x,y) and testing (X,Y) 'question_body' datasets.  test_size default 0.25, random_state for shuffling data
xTrain,xTest,yTrain, yTest = train_test_split(training_set['question_body'], yTrain, test_size=0.25,random_state=30)

tokenizer = Tokenizer(num_words = maxFeatures) # setup the tokenizer, num words in question_body to vectorize
tokenizer.fit_on_texts(list(xTrain)) 

xVal = xTest # set xVal to xTest
xVal = tokenizer.texts_to_sequences(xVal) # transform xVal into sequence of integers

xTrain = tokenizer.texts_to_sequences(xTrain) # transform xTrain into sequence of integers

xTrain = pad_sequences(xTrain, maxlen = maxLength) # pad sequence so that the vectors can have the same lengths
xVal   = pad_sequences(xVal, maxlen = maxLength) # pad sequence so that the vectors can have the same lengths

yVal = yTest

print("PADDED AND TOKENIZED SEQUENCES, ALL VECTORS HAVE THE SAME LENGTH")
print("----------------------------------------------------------------")

print("xTrain Sequence, Padded and Tokenized: ", xTrain.shape)
print("yTrain                               : ", yTrain.shape)

print("xVal Sequence, Padded and Tokenized  : ", xVal.shape)
print("yVal                                 : ", yVal.shape)


model.fit(xTrain, yTrain, batch_size=128, epochs=5, verbose=2, validation_data = (xVal,yVal))

PADDED AND TOKENIZED SEQUENCES, ALL VECTORS HAVE THE SAME LENGTH
----------------------------------------------------------------
xTrain Sequence, Padded and Tokenized:  (4559, 1000)
yTrain                               :  (4559,)
xVal Sequence, Padded and Tokenized  :  (1520, 1000)
yVal                                 :  (1520,)
Epoch 1/5
36/36 - 141s - loss: 1.3223 - accuracy: 0.4242 - val_loss: 1.0491 - val_accuracy: 0.6092 - 141s/epoch - 4s/step
Epoch 2/5
36/36 - 142s - loss: 0.7331 - accuracy: 0.8111 - val_loss: 0.7154 - val_accuracy: 0.7822 - 142s/epoch - 4s/step
Epoch 3/5
36/36 - 148s - loss: 0.3505 - accuracy: 0.9504 - val_loss: 0.5099 - val_accuracy: 0.8414 - 148s/epoch - 4s/step
Epoch 4/5
36/36 - 135s - loss: 0.1544 - accuracy: 0.9833 - val_loss: 0.4848 - val_accuracy: 0.8428 - 135s/epoch - 4s/step
Epoch 5/5
36/36 - 131s - loss: 0.0770 - accuracy: 0.9947 - val_loss: 0.4591 - val_accuracy: 0.8553 - 131s/epoch - 4s/step


In [29]:
model = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed")
training_set['question_title'] = training_set['question_title'].apply(mineText)

question = [model([training_set.iloc[i].question_title])[0] for i in range(training_set.shape[0])]


In [33]:
def sentimentSimilarity(query):
    query = model([query])
    similarity = cosine_similarity(question, query)
    similarityVal = [similarity[i][0] for i in range(similarity.shape[0])]
    return np.argmax(similarityVal)

In [36]:
sample_question = 'I am having an issue with accessing different OpenCV libraries. What are some functions that may help solve my issue?'
print('SAMPLE QUESTION')
print("-----------------------------")
print(sample_question)

print('\nPREPROCESSED QUESTION')
print("-----------------------------")
sample_question = mineText(sample_question)
print(sample_question)



SAMPLE QUESTION
-----------------------------
I am having an issue with accessing different OpenCV libraries. What are some functions that may help solve my issue?

PREPROCESSED QUESTION
-----------------------------
issue accessing different opencv library function may help solve issue 
